In [ ]:
## Automated downloading of  plot geoshapes from the https://parselsorgu.tkgm.gov.tr/ website. It takes in a csv with the village, land and parcel code ,
## creates a url , retrieves the json response . The farmer ID , village, land and parcel codes are provided by UTZ. We convert the village 
## name to village code using the parselsorgu website.
## The URL structure is https://megsisapi.tkgm.gov.tr/api/parsel/<villagecode>/<landcode>/<parcelcode>
## The json response is addded is added back to the dataframe and a csv is generated 
## The website allows about 150 hits from one ip per day. So we need to break up the original dataset in chunks of 150 records 
## and use vpn to change the ip address

In [ ]:
import requests
import pandas as pd
import json
import numpy as np
from collections import OrderedDict
import pprint as pp

filename='Batches/T4200-' 
oCsv=pd.read_csv(filename+'.csv')
oCsv

In [ ]:
shape=[]
parcelA=[]
urlA=[]
def setData(urlString,json,parcel):
    urlA.append(urlString)
    shape.append(json)
    parcelA.append(parcel)
    print(urlString,'parcel>>>>',parcel)
for index,row  in oCsv.iterrows():
    print(index)
    url='https://megsisapi.tkgm.gov.tr/api/parsel/'+str(row['Code'])+'/'+str(row['Land'])+'/'+str(row['Parcel'])
    response = requests.get(url)

    try:
        response.raise_for_status()
        json_obj = response.json()
        json_obj['properties']['Farmer']=row['Farmer Code']
        parcel=json_obj['properties']['adaNo']+'/'+json_obj['properties']['parselNo']
        setData(response.url,json_obj,parcel)
    except requests.exceptions.HTTPError as e:
         setData(response.url,' ',' ')

oCsv['parcel']=parcelA
oCsv['URL']=urlA
oCsv['shape']=shape
df=pd.DataFrame(oCsv,columns=oCsv.keys())
newfName=filename+'-done.csv'
df.to_csv(newfName,index=False)  
print("DONE")